<a href="https://colab.research.google.com/github/bilalercin/submarine-waste-detection/blob/main/ai_final03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# AI_PROJE klasörünüze gidin (MyDrive altındaysa)
%cd /content/drive/MyDrive/AI_PROJE/dataset
!ls

[Errno 2] No such file or directory: '/content/drive/MyDrive/AI_PROJE/dataset'
/content
sample_data


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import os, json
from tqdm import tqdm

def coco_to_yolo(json_path, img_dir, out_label_dir):
    os.makedirs(out_label_dir, exist_ok=True)
    with open(json_path, 'r') as f:
        coco = json.load(f)
    # kategori id → sıfır-tabanlı index
    cat_map = {cat['id']: i for i,cat in enumerate(coco['categories'])}
    # image_id → dosya adı, boyut bilgisi
    img_info = {img['id']:{
                    'file_name':img['file_name'],
                    'width': img['width'],
                    'height': img['height']
                } for img in coco['images']}

    anns_per_image = {img_id: [] for img_id in img_info}
    for ann in coco['annotations']:
        anns_per_image[ann['image_id']].append(ann)

    for img_id, anns in tqdm(anns_per_image.items()):
        info = img_info[img_id]
        fname = os.path.splitext(info['file_name'])[0] + '.txt'
        label_path = os.path.join(out_label_dir, fname)
        lines = []
        for ann in anns:
            x,y,w,h = ann['bbox']
            # normalize
            x_c = (x + w/2) / info['width']
            y_c = (y + h/2) / info['height']
            w_n = w / info['width']
            h_n = h / info['height']
            cls = cat_map[ann['category_id']]
            lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}")
        with open(label_path, 'w') as f:
            f.write("\n".join(lines))

# Yol tanımları
BASE = '/content/drive/MyDrive/AI_PROJE/dataset'
ANN = os.path.join(BASE, 'instance_version')
IMG = os.path.join(BASE, 'images')

# train
coco_to_yolo(os.path.join(ANN,'instances_train_trashcan.json'),
            os.path.join(IMG,'train'),
            os.path.join(BASE,'labels/train'))
# val
coco_to_yolo(os.path.join(ANN,'instances_val_trashcan.json'),
            os.path.join(IMG,'val'),
            os.path.join(BASE,'labels/val'))


100%|██████████| 1147/1147 [00:11<00:00, 102.96it/s]


In [ ]:
import yaml
with open(os.path.join(ANN,'instances_train_trashcan.json')) as f:
    cats = json.load(f)['categories']
data = {
    'train': os.path.join(BASE,'images/train'),
    'val'  : os.path.join(BASE,'images/val'),
    'nc'   : len(cats),
    'names': [c['name'] for c in cats]
}
with open('data.yaml','w') as f:
    yaml.dump(data, f, sort_keys=False)
print(open('data.yaml').read())


train: /content/drive/MyDrive/AI_PROJE/dataset/images/train
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val
nc: 22
names:
- rov
- plant
- animal_fish
- animal_starfish
- animal_shells
- animal_crab
- animal_eel
- animal_etc
- trash_clothing
- trash_pipe
- trash_bottle
- trash_bag
- trash_snack_wrapper
- trash_can
- trash_cup
- trash_container
- trash_unknown_instance
- trash_branch
- trash_wreckage
- trash_tarp
- trash_rope
- trash_net



In [ ]:
!yolo detect train \
    data=data.yaml \
    model=yolov8n.pt \
    epochs=50 \
    imgsz=640 \
    batch=16 \
    project='runs/trashcan' \
    name='yolov8_trashcan'


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.140 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, l

In [ ]:
from ultralytics import YOLO
import pandas as pd

# 1. En iyi model ağırlıklarını yükleyin
model = YOLO('/content/drive/MyDrive/AI_PROJE/dataset/runs/trashcan/yolov8_trashcan2/weights/best.pt')

# 2. Validasyon setinde çalıştırın
results = model.val(data='/content/drive/MyDrive/AI_PROJE/dataset/data.yaml')

# 3. Metrik objesine erişin
det = results.box

# 4. Sınıf isimleri
names = results.names  # data.yaml içindeki names listesi

# 5. Raporu oluşturun
report = []
for i, cls_name in enumerate(names):
    p, r, ap50, ap = det.class_result(i)  # precision, recall, AP@0.5, mAP@0.5-0.95
    report.append({
        'class': cls_name,
        'Precision': round(p, 3),
        'Recall':    round(r, 3),
        'AP@0.50':   round(ap50, 3),
        'mAP@0.50-0.95': round(ap, 3)
    })

# 6. DataFrame’e çevirip yazdırın
df = pd.DataFrame(report)
print(df.to_string(index=False))


Ultralytics 8.3.140 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,938 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 21.7MB/s]


val: Fast image access ✅ (ping: 0.6±0.1 ms, read: 0.0±0.0 MB/s, size: 20.3 KB)


val: Scanning /content/drive/MyDrive/AI_PROJE/dataset/labels/val.cache... 1147 images, 0 backgrounds, 52 corrupt: 100%|██████████| 1147/1147 [00:00<?, ?it/s]

val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame0000054.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0037]
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame0000056.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0021      1.0037      1.0037]
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame0000058.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0037      1.0037]
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame0000060.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0021      1.0037]
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame0000062.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0021      1.0037]
val: /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000052_frame000


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 69/69 [00:24<00:00,  2.83it/s]


                   all       1095       2405      0.767      0.607      0.665       0.49
                   rov        486        631      0.891       0.91      0.945       0.82
                 plant         90         99      0.768      0.535      0.633      0.379
           animal_fish        115        152      0.686      0.368      0.522      0.258
       animal_starfish         35        101      0.879      0.573      0.608      0.348
         animal_shells         35         61       0.58      0.567      0.535      0.307
           animal_crab         34         63      0.187      0.365      0.264      0.131
            animal_eel         77         84      0.619       0.69      0.678      0.405
            animal_etc         55         63       0.79      0.477      0.622      0.377
        trash_clothing         17         17      0.928      0.353      0.576      0.519
            trash_pipe         39         39      0.926      0.949      0.937      0.834
          trash_bottl

In [ ]:
from ultralytics import YOLO
import yaml, glob, os
import numpy as np
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score

def compute_iou(box1, box2):
    # box = [x1, y1, x2, y2]
    x_left   = max(box1[0], box2[0])
    y_top    = max(box1[1], box2[1])
    x_right  = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    inter_area  = (x_right - x_left) * (y_bottom - y_top)
    box1_area   = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area   = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (box1_area + box2_area - inter_area + 1e-6)

# 1. Modeli ve data.yaml’ı yükleyin
model = YOLO('/content/drive/MyDrive/AI_PROJE/dataset/runs/trashcan/yolov8_trashcan2/weights/best.pt')
with open('/content/drive/MyDrive/AI_PROJE/dataset/data.yaml') as f:
    data_cfg = yaml.safe_load(f)
val_dir = data_cfg['val']  # örn. 'images/val'

# 2. Resim ve etiket yollarını toplayın
img_paths = sorted(glob.glob(os.path.join(val_dir, '*.*')))

y_true, y_pred = [], []
bg_label = len(model.names)  # “no_object” için özel ID
class_names = [model.names[i] for i in range(len(model.names))] + ['no_object']

# 3. Her resim için
for img_path in img_paths:
    # -- Ground truth’u oku
    lbl_path = img_path.replace('/images/', '/labels/').rsplit('.',1)[0] + '.txt'
    gt_boxes, gt_classes = [], []
    if os.path.exists(lbl_path):
        with open(lbl_path) as f:
            for line in f:
                cls, x, y, w, h = map(float, line.split())
                im_w, im_h      = Image.open(img_path).size
                # normalize xywh -> x1,y1,x2,y2
                x1 = (x - w/2) * im_w
                y1 = (y - h/2) * im_h
                x2 = (x + w/2) * im_w
                y2 = (y + h/2) * im_h
                gt_boxes.append([x1, y1, x2, y2])
                gt_classes.append(int(cls))

    # -- Model tahminlerini al
    res = model.predict(source=img_path, imgsz=640, conf=0.25)[0]
    preds = res.boxes.data.cpu().numpy()  # [x1,y1,x2,y2,conf,cls]
    pred_boxes   = [p[:4] for p in preds]
    pred_classes = preds[:,5].astype(int)

    # 4. Eşleştirme
    gt_matched   = [False] * len(gt_boxes)
    pred_matched = [False] * len(pred_boxes)

    # 4a. True Positive’ler
    for i, pb in enumerate(pred_boxes):
        best_iou, best_j = 0, -1
        for j, gb in enumerate(gt_boxes):
            iou = compute_iou(pb, gb)
            if iou > best_iou:
                best_iou, best_j = iou, j
        if best_iou >= 0.5:
            y_true.append(gt_classes[best_j])
            y_pred.append(pred_classes[i])
            gt_matched[best_j]   = True
            pred_matched[i]      = True

    # 4b. False Positives (modelin boş yere buldukları)
    for i, matched in enumerate(pred_matched):
        if not matched:
            y_true.append(bg_label)
            y_pred.append(pred_classes[i])

    # 4c. False Negatives (modelin kaçırdıkları)
    for j, matched in enumerate(gt_matched):
        if not matched:
            y_true.append(gt_classes[j])
            y_pred.append(bg_label)

# 5. Accuracy ve Classification Report
acc = accuracy_score(y_true, y_pred)
print(f"Overall Accuracy: {acc:.3f}\n")

print("Classification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    labels       = list(range(len(class_names))),
    target_names = class_names,
    zero_division=0
))



image 1/1 /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000002_frame0000013.jpg: 384x640 2 trash_unknown_instances, 39.0ms
Speed: 4.1ms preprocess, 39.0ms inference, 12.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000002_frame0000014.jpg: 384x640 1 trash_bag, 1 trash_unknown_instance, 30.9ms
Speed: 7.6ms preprocess, 30.9ms inference, 12.1ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000002_frame0000015.jpg: 384x640 1 trash_unknown_instance, 19.5ms
Speed: 2.6ms preprocess, 19.5ms inference, 8.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_000002_frame0000016.jpg: 384x640 1 trash_unknown_instance, 39.7ms
Speed: 4.5ms preprocess, 39.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/AI_PROJE/dataset/images/val/vid_00